In [1]:
import tensorflow as tf
import numpy as np
import os
import glob
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import LearningRateScheduler
from PIL import Image
from tensorflow.keras.layers import LeakyReLU
import matplotlib.pyplot as plt
from rnnconv import RnnConv

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
(train_images2, train_labels2), (test_images2, test_labels2) = datasets.cifar100.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0
train_images2, test_iamges2 = train_images2 / 255.0, test_images / 255.0

train_images = np.concatenate((train_images, train_images2) ,axis=0)
test_images = np.concatenate((test_images, test_images2), axis=0)

train_labels = np.concatenate((train_labels, train_labels2), axis=0)
test_labels = np.concatenate((test_labels, test_labels2), axis=0)

In [2]:
print(tf.config.list_physical_devices("GPU"))
iterations = 1

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
with open('D:\\\data\\data.npy', 'rb') as f:
    data = (np.load(f) / 255.0).reshape((99999,32,32,3))

In [4]:
train_images = np.concatenate((train_images, data), axis=0)

In [5]:
model = tf.keras.Sequential() 
relu_amount = 0.01

for i in range(iterations):

    model.add(layers.Conv2D(filters=64, kernel_size=3, input_shape=(32, 32, 3), strides = (2, 2), padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))

    model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
    model.add(layers.ConvLSTM2D(filters=256, kernel_size=3, strides=(2, 2),   padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))
    
    model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
    model.add(layers.ConvLSTM2D(filters=512, kernel_size=3, strides=(2, 2),   padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))
    
    model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
    model.add(layers.ConvLSTM2D(filters=512, kernel_size=3, strides=(2, 2),   padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))

    model.add(layers.Conv2D(filters=32, kernel_size=1,  strides = (1,1), padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))

    model.add(layers.Conv2D(filters=512, kernel_size=1,  strides = (1,1), padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))
    
    model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
    model.add(layers.ConvLSTM2D(filters=512, kernel_size=2, strides=(1, 1), padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))
    model.add(layers.Lambda(lambda x:tf.nn.depth_to_space(x,2,data_format='NHWC')))

    model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
    model.add(layers.ConvLSTM2D(filters=512, kernel_size=3, strides=(1, 1), padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))
    model.add(layers.Lambda(lambda x:tf.nn.depth_to_space(x,2,data_format='NHWC')))
    
    
    model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
    model.add(layers.ConvLSTM2D(filters=256, kernel_size=3, strides=(1, 1), padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))
    model.add(layers.Lambda(lambda x:tf.nn.depth_to_space(x,2,data_format='NHWC')))

    model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
    model.add(layers.ConvLSTM2D(filters=128, kernel_size=3, strides=(1, 1), padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))
    model.add(layers.Lambda(lambda x:tf.nn.depth_to_space(x,2,data_format='NHWC')))

    model.add(layers.Conv2D(filters=3, kernel_size=1, strides = (1,1), padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))

In [6]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.9, epsilon=1e-06, amsgrad=True)

def SSIMLoss(y_true, y_pred):
    return tf.math.square(1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.0)))

model.compile(optimizer=opt, loss=SSIMLoss, metrics=[SSIMLoss])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 16, 64)        1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 16, 16, 64)        0         
_________________________________________________________________
lambda (Lambda)              (None, 1, 16, 16, 64)     0         
_________________________________________________________________
conv_lst_m2d (ConvLSTM2D)    (None, 8, 8, 256)         2950144   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 8, 8, 256)         0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 1, 8, 8, 256)      0         
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 4, 4, 512)         1

In [7]:
from skimage.util import random_noise

# noisy_image = random_noise(train_images, mode="s&p", amount=0.01)
validation_data = (train_images[2000:10000,:,:,:],train_images[2000:10000,:,:,:])

In [8]:
train_images_ = tf.convert_to_tensor(train_images.reshape((-1,3,32,32)))

# reshaped_images = tf.expand_dims(train_images, axis=1)
model3 = model.fit(train_images, train_images, batch_size=32, epochs=20, validation_split=0.05)

Epoch 1/20
5938/5938 [==============================] - 811s 137ms/step - loss: 0.1067 - SSIMLoss: 0.1067 - val_loss: 0.0626 - val_SSIMLoss: 0.0626
Epoch 2/20
5938/5938 [==============================] - 802s 135ms/step - loss: 0.0401 - SSIMLoss: 0.0401 - val_loss: 0.0528 - val_SSIMLoss: 0.0528
Epoch 3/20
5938/5938 [==============================] - 806s 136ms/step - loss: 0.0367 - SSIMLoss: 0.0367 - val_loss: 0.0516 - val_SSIMLoss: 0.0516
Epoch 4/20
5938/5938 [==============================] - 802s 135ms/step - loss: 0.0349 - SSIMLoss: 0.0349 - val_loss: 0.0478 - val_SSIMLoss: 0.0478
Epoch 5/20
5938/5938 [==============================] - 808s 136ms/step - loss: 0.0330 - SSIMLoss: 0.0330 - val_loss: 0.0468 - val_SSIMLoss: 0.0468
Epoch 6/20
5938/5938 [==============================] - 806s 136ms/step - loss: 0.0323 - SSIMLoss: 0.0323 - val_loss: 0.0458 - val_SSIMLoss: 0.0458
Epoch 7/20
3612/5938 [=================>............] - ETA: 5:11 - loss: 0.0314 - SSIMLoss: 0.0314

KeyboardInterrupt: 

In [ ]:
tf.expand_dims(train_images, axis=1).shape

In [ ]:
from matplotlib import pyplot as plt

img = train_images_[400]
img_ = img.numpy().reshape((32,32,3))
plt.imshow(img_, interpolation='nearest')
print(img_.shape)
plt.show()

In [ ]:
# img_pred = test_images[0:10].reshape((10,3,32,32,))
full = model.predict(test_images[20:30,:,:,:])

In [ ]:
plt.plot(model3.history["val_SSIMLoss"])

In [ ]:
# img_pred_full= img_pred.reshape((10,32,32,3))

plt.figure(figsize=(10,10))
for i in range(10):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_images[20:30,:,:,:][i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
plt.show()

In [ ]:
full_pred = full.reshape((10,32,32,3))

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(10):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(full_pred[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
plt.show()

In [ ]:
full_pred[0]

In [ ]:
import numpy as np
image_number = np.random.randint(low=0, high=train_images.shape[0], size=1)[0]

In [ ]:
plt.imshow(train_images[image_number])

In [ ]:
train_images[image_number].shape

### Loading Data

In [ ]:
data_url = 'D:\\data\\test_challenge_2018'

In [ ]:
class ImageLoader:
    
    def __init__(self, image_directory):
        
        self.image_directory = image_directory
        self.image_paths = [os.path.abspath(os.path.join(self.image_directory, p)) for p in os.listdir(self.image_directory) if p.endswith(('jpg', 'png'))]
        
        print("There are {} iamges".format(len(self.image_paths)))
        
        self.num_images = len(self.image_paths)
        self.process_images(self.num_images)
        
    def process_images(self, num_images):
        
        self.data = np.array([])
        
        for i in range(num_images):
            print("Processed image {}".format(i))
            
            image_filename = "image%s.png" % i
            
            PIL_image = Image.open(self.image_paths[i]).resize((32, 32), Image.ANTIALIAS)
            
            self.data = np.append(self.data, np.array(PIL_image))

loader = ImageLoader(data_url)
data = loader.data

In [ ]:
with open('D:\\data\\data.npy', 'wb') as f:
    np.save(f, data)

In [ ]:
picture = train_images[32740]
plt.imshow(picture)

In [ ]:
! pip install scikit-image